In [7]:
import scipy.stats as stats
import pandas as pd
# Load your dataset
data = pd.read_csv('../data/raw/MachineLearningRating_v3.txt', sep='|')


# Recalculate LossRatio if not already done
data['LossRatio'] = data['TotalClaims'] / data['TotalPremium']

# Drop missing LossRatio values
anova_data = data.dropna(subset=['LossRatio'])

# Perform ANOVA
groups = [group['LossRatio'].values for name, group in anova_data.groupby('Province')]
f_stat, p_val = stats.f_oneway(*groups)

print(f"ANOVA F-statistic: {f_stat:.4f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Reject H0: Significant risk difference across provinces.")
else:
    print("Fail to reject H0: No significant risk difference across provinces.")


/var/folders/pp/3nx806hs7bl388g7c6clvc5h0000gn/T/ipykernel_37322/2308676876.py:4: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/raw/MachineLearningRating_v3.txt', sep='|')


ANOVA F-statistic: nan, p-value: nan
Fail to reject H0: No significant risk difference across provinces.


/Users/oza/Developer/10 Acadamy/week3/insurance-risk-analytics/env/lib/python3.13/site-packages/numpy/_core/_methods.py:134: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [8]:
gender_data = data.dropna(subset=['LossRatio', 'Gender'])
male_loss = gender_data[gender_data['Gender']=='Male']['LossRatio']
female_loss = gender_data[gender_data['Gender']=='Female']['LossRatio']

t_stat, p_val = stats.ttest_ind(male_loss, female_loss, equal_var=False)

print(f"T-statistic: {t_stat:.4f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Reject H0: Significant risk difference between genders.")
else:
    print("Fail to reject H0: No significant risk difference between genders.")


T-statistic: nan, p-value: nan
Fail to reject H0: No significant risk difference between genders.


/Users/oza/Developer/10 Acadamy/week3/insurance-risk-analytics/env/lib/python3.13/site-packages/scipy/stats/_stats_py.py:6154: RuntimeWarning: invalid value encountered in scalar subtract
  d = mean1 - mean2
/Users/oza/Developer/10 Acadamy/week3/insurance-risk-analytics/env/lib/python3.13/site-packages/scipy/stats/_stats_py.py:6707: RuntimeWarning: invalid value encountered in scalar subtract
  estimate = m1 - m2
